In [1]:
import pandas as pd
import numpy as np
import pmdarima as pm
import warnings
warnings.filterwarnings("ignore")

from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('sell-in.txt',sep='\t')
df['fecha'] = pd.to_datetime(df['periodo'].astype(str), format='%Y%m')
df.head()

,periodo,customer_id,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn,fecha
0,201701,10234,20524,0,2,0.05300,0.05300,2017-01-01
1,201701,10032,20524,0,1,0.13628,0.13628,2017-01-01
2,201701,10217,20524,0,1,0.03028,0.03028,2017-01-01
3,201701,10125,20524,0,1,0.02271,0.02271,2017-01-01
4,201701,10012,20524,0,11,1.54452,1.54452,2017-01-01


In [3]:
df = df[df['fecha'] <= '2018-01-01']

In [4]:
predecir = pd.read_csv('productos_a_predecir.txt', sep='\t')
predecir.head()

,product_id
0,20001
1,20002
2,20003
3,20004
4,20005


In [5]:
fechas = pd.date_range(start='2017-01-01', end='2019-12-01', freq='MS') #pd.date_range(start='2017-01-01', periods=36, freq='M')
df_fechas = pd.DataFrame({'fecha': fechas})

In [6]:
def serie_producto(id_producto):
    df_prod = df[df['product_id']==id_producto].loc[:, ['fecha','tn']].groupby('fecha').agg({'tn': 'sum'})
    df_ret = pd.merge(df_fechas, df_prod, on='fecha', how='left')
    df_ret['tn'].interpolate(method='linear', inplace=True)
    df_ret.fillna(0)

    #print(f"Producto {id_producto}, lineas: {len(df_prod)}")
    return df_ret

a = serie_producto(20089)

In [7]:
df_pred = predecir
df_pred['tn'] = 0

In [8]:
pred = []
pred_final = None

In [9]:
for i in range(len(predecir)):
    id_producto = predecir.iloc[i,0]
    df_prod = serie_producto(id_producto)
    
    print(f"Progress: {i+1}/{len(predecir)}", end="\r")

    if len(df_prod==24):
        try:
            #modelo = SARIMAX(df_prod['tn'], order=(2, 1, 1))
            modelo = pm.auto_arima(df_prod['tn'], seasonal=True, m=12, suppress_warnings=True)
            #modelo_fit = modelo.fit()

            periodos_futuros = 2
            pred = modelo.predict(steps=periodos_futuros)[37]
            #pred = modelo_fit.forecast(steps=periodos_futuros)
            #pred_final = pred.iloc[-1]
            pred_final = pred
            #print(f"Producto : {id_producto} pred: {pred} predfinal: {pred_final} modelo {print(modelo)}")
            df_pred.iloc[i,1] = pred_final
        except:
            next

In [ ]:
df_pred

,product_id,tn
0,20001,0
1,20002,0
2,20003,0
3,20004,0
4,20005,0
...,...,...
775,21263,0
776,21265,0
777,21266,0
778,21267,0


In [ ]:
df_pred.to_csv('pred_autoarima.csv', index=False)